## Notes
- 215서버 가상환경 bkms에서 작업

# 230604 Update

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize, sent_tokenize
import torch
from tqdm.notebook import tqdm
import psycopg2
import psycopg2.extras as extras

In [4]:
# server table 확인
connection_info = "host=147.47.200.145 dbname=teamdb16 user=team16 password=qwer16 port=34543"
conn = psycopg2.connect(connection_info)

try:
    # 테이블을 Pandas.Dataframe으로 추출
    server_df = pd.read_sql('SELECT * FROM citation_data',conn)

except psycopg2.Error as e:
    # 데이터베이스 에러 처리
    print("DB error: ", e)
    
finally:
    # 데이터베이스 연결 해제 필수!!
    conn.close()

/tmp/ipykernel_1051318/3039801870.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  server_df = pd.read_sql('SELECT * FROM citation_data',conn)


In [5]:
server_df

,id,title,venue,year,keywords,fos,refer_doc,n_citation,doc_type,abstract
0,53e9af94b7602d97039dd8b5,A Situation Calculus-based Approach To Model U...,Computing Research Repository,2003,"{""markov property"",satisfiability,""situation c...",Machine learning,"{53e99a3cb7602d970229d140,53e9a31fb7602d9702c2...",11,Journal,This paper presents an augmented situation c...
1,53e9af94b7602d97039ddb53,A User Profiling Architecture for Textual-Base...,"Inteligencia Artificial, Revista Iberoamerican...",2003,"{personalization,""intelligent agents"",""user pr...",Machine learning,"{53e9ab20b7602d97034a5c28,53e999cab7602d970220...",11,Journal,Several intelligent agents have been developed...
2,53e9af94b7602d97039dd707,Mining fuzzy association rules from questionna...,Knowl.-Based Syst.,2009,"{""fuzzy sets"",""data mining"",""questionnaire dat...",Machine learning,"{5582cbfd0cf2a81de26651ca,53e99915b7602d970214...",98,Journal,Association rule mining is one of most popular...
3,53e9af94b7602d97039ddc91,An Integrative Neural Network with Feedback Co...,IJCNN,2006,"{""learning (artificial intelligence)"",""pattern...",Machine learning,"{53e9aa9cb7602d9703415186,53e99b51b7602d97023f...",2,Conference,This paper presents a novel integrative neural...
4,53e9af94b7602d97039ddb9a,Data mining for intelligent structure form sel...,PAKDD Workshops,2007,"{""high rise case base"",""fuzzy interval divisio...",Machine learning,{53e9a7eab7602d9703128855},1,Conference,This paper presents a uniform model for high-r...
...,...,...,...,...,...,...,...,...,...,...
644795,53e9af94b7602d97039dcb2a,An alternative approach for modelling and simu...,Simulation Modelling Practice and Theory,2004,"{""Neural network"",""Traffic model problem"",Iden...",Machine learning,{53e99991b7602d97021d2b83},30,Journal,It is assumed that there is a complicated rela...
644796,53e9af94b7602d97039dc83e,Fuzzy linear and nonlinear discriminant analys...,Intell. Data Anal.,2009,"{""fuzzy logic"",""nonlinear discriminant analysi...",Machine learning,"{53e9ad33b7602d970371729e,53e9990db7602d970214...",2,Journal,This paper presents two novels algorithms base...
644797,53e9af94b7602d97039dce17,A visual framework to understand similarity qu...,IJBIDM,2010,"{""new proposed structure"",""share dynamic"",""exi...",Machine learning,"{53e9a8a9b7602d97031f6116,53e998e9b7602d970212...",2,Journal,This paper presents the MAMView framework to h...
644798,53e9af94b7602d97039dce6f,A multi-level thresholding-based method to lea...,CIARP,2011,"{""data warehouse"",""fuzzy measure"",""genetic alg...",Machine learning,"{558ac63284ae84d265bf8f5f,53e99a4eb7602d97022a...",0,Conference,Learn fuzzy membership functions automatically...


In [52]:
ids = server_df['id']

In [56]:
for i in ids:
    if len(i) != 24:
        print(i)

In [8]:
abstracts = list(server_df['abstract'])

In [9]:
# abstract가 nan인 row
for i, abstract in enumerate(abstracts):
    if not isinstance(abstract, str):
        nan_idx.append(i)
        print(i)
        print(abstract)

In [10]:
from simcse import SimCSE
model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")

In [ ]:
%%time
embedding_list = []
# for i, abstract in tqdm(enumerate(abstracts)):
for i, abstract in enumerate(tqdm(abstracts)):
    try:
        sent_emb_list = model.encode(sent_tokenize(abstract), device="cuda:1", max_length=256)
        embedding_list.append(torch.mean(sent_emb_list, dim=0)) # 각 문장의 임베딩의 평균 (torch.Size([768]))
    except:
        print(f'Error: {i}')
        print(abstract)
        embedding_list.append('dummy')

  0%|          | 0/644800 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  1.08it/s]

100%|██████████| 1/1 [00:00<00:00, 131.30it/s]

100%|██████████| 1/1 [00:00<00:00, 97.73it/s]

100%|██████████| 1/1 [00:00<00:00, 116.41it/s]

100%|██████████| 1/1 [00:00<00:00, 123.89it/s]

100%|██████████| 1/1 [00:00<00:00, 123.09it/s]

100%|██████████| 1/1 [00:00<00:00, 97.77it/s]

100%|██████████| 1/1 [00:00<00:00, 139.59it/s]

100%|██████████| 1/1 [00:00<00:00, 138.41it/s]

100%|██████████| 1/1 [00:00<00:00, 138.65it/s]

100%|██████████| 1/1 [00:00<00:00, 143.86it/s]

100%|██████████| 1/1 [00:00<00:00, 141.66it/s]

100%|██████████| 1/1 [00:00<00:00, 35.62it/s]

100%|██████████| 1/1 [00:00<00:00, 139.76it/s]

100%|██████████| 1/1 [00:00<00:00, 146.01it/s]

100%|██████████| 1/1 [00:00<00:00, 138.90it/s]

100%|██████████| 1/1 [00:00<00:00, 140.78it/s]

100%|██████████| 1/1 [00:00<00:00, 148.16it/s]

100%|██████████| 1/1 [00:00<00:00, 137.97it/s]

100%|██████████| 1/1 [00:00<00:00, 137.95it/s]

100%|██████████| 1/1 [00:00<00:00, 144.76it


100%|██████████| 1/1 [00:00<00:00, 142.02it/s]

100%|██████████| 1/1 [00:00<00:00, 134.26it/s]

100%|██████████| 1/1 [00:00<00:00, 145.02it/s]

100%|██████████| 1/1 [00:00<00:00, 144.33it/s]

100%|██████████| 1/1 [00:00<00:00, 138.65it/s]

100%|██████████| 1/1 [00:00<00:00, 127.00it/s]

100%|██████████| 1/1 [00:00<00:00, 99.48it/s]

100%|██████████| 1/1 [00:00<00:00, 133.76it/s]

100%|██████████| 1/1 [00:00<00:00, 133.95it/s]

100%|██████████| 1/1 [00:00<00:00, 135.42it/s]

100%|██████████| 1/1 [00:00<00:00, 136.93it/s]

100%|██████████| 1/1 [00:00<00:00, 138.62it/s]

100%|██████████| 1/1 [00:00<00:00, 137.98it/s]

100%|██████████| 1/1 [00:00<00:00, 139.05it/s]

100%|██████████| 1/1 [00:00<00:00, 135.23it/s]

100%|██████████| 1/1 [00:00<00:00, 125.13it/s]

100%|██████████| 1/1 [00:00<00:00, 138.07it/s]

100%|██████████| 1/1 [00:00<00:00, 143.06it/s]

100%|██████████| 1/1 [00:00<00:00, 133.29it/s]

100%|██████████| 1/1 [00:00<00:00, 136.86it/s]

100%|██████████| 1/1 [00:00<00:00, 133.2

100%|██████████| 1/1 [00:00<00:00, 143.03it/s]

100%|██████████| 1/1 [00:00<00:00, 147.01it/s]

100%|██████████| 1/1 [00:00<00:00, 142.08it/s]

100%|██████████| 1/1 [00:00<00:00, 148.38it/s]

100%|██████████| 1/1 [00:00<00:00, 149.04it/s]

100%|██████████| 1/1 [00:00<00:00, 130.51it/s]

100%|██████████| 1/1 [00:00<00:00, 99.62it/s]

100%|██████████| 1/1 [00:00<00:00, 134.27it/s]

100%|██████████| 1/1 [00:00<00:00, 145.30it/s]

100%|██████████| 1/1 [00:00<00:00, 133.39it/s]

100%|██████████| 1/1 [00:00<00:00, 128.91it/s]

100%|██████████| 1/1 [00:00<00:00, 144.59it/s]

100%|██████████| 1/1 [00:00<00:00, 144.52it/s]

100%|██████████| 1/1 [00:00<00:00, 136.99it/s]

100%|██████████| 1/1 [00:00<00:00, 143.47it/s]

100%|██████████| 1/1 [00:00<00:00, 159.26it/s]

100%|██████████| 1/1 [00:00<00:00, 140.62it/s]

100%|██████████| 1/1 [00:00<00:00, 139.48it/s]

100%|██████████| 1/1 [00:00<00:00, 146.39it/s]

100%|██████████| 1/1 [00:00<00:00, 145.68it/s]

100%|██████████| 1/1 [00:00<00:00, 99.61i

100%|██████████| 1/1 [00:00<00:00, 120.19it/s]

100%|██████████| 1/1 [00:00<00:00, 117.95it/s]

100%|██████████| 1/1 [00:00<00:00, 121.24it/s]

100%|██████████| 1/1 [00:00<00:00, 114.41it/s]

100%|██████████| 1/1 [00:00<00:00, 88.48it/s]

100%|██████████| 1/1 [00:00<00:00, 133.61it/s]

100%|██████████| 1/1 [00:00<00:00, 126.53it/s]

100%|██████████| 1/1 [00:00<00:00, 138.22it/s]

100%|██████████| 1/1 [00:00<00:00, 143.53it/s]

100%|██████████| 1/1 [00:00<00:00, 145.74it/s]

100%|██████████| 1/1 [00:00<00:00, 141.40it/s]

100%|██████████| 1/1 [00:00<00:00, 131.95it/s]

100%|██████████| 1/1 [00:00<00:00, 143.28it/s]

100%|██████████| 1/1 [00:00<00:00, 114.08it/s]

100%|██████████| 1/1 [00:00<00:00, 133.03it/s]

100%|██████████| 1/1 [00:00<00:00, 103.53it/s]

100%|██████████| 1/1 [00:00<00:00, 145.72it/s]

100%|██████████| 1/1 [00:00<00:00, 136.52it/s]

100%|██████████| 1/1 [00:00<00:00, 144.50it/s]

100%|██████████| 1/1 [00:00<00:00, 139.25it/s]

100%|██████████| 1/1 [00:00<00:00, 140.56

...

100%|██████████| 1/1 [00:00<00:00, 140.64it/s]

100%|██████████| 1/1 [00:00<00:00, 137.23it/s]

100%|██████████| 1/1 [00:00<00:00, 140.72it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100%|██████████| 1/1 [00:00<00:00, 129.15it/s]

100%|██████████| 1/1 [00:00<00:00, 135.57it/s]

100%|██████████| 1/1 [00:00<00:00, 140.44it/s]

100%|██████████| 1/1 [00:00<00:00, 138.90it/s]

100%|██████████| 1/1 [00:00<00:00, 131.98it/s]

100%|██████████| 1/1 [00:00<00:00, 145.12it/s]

100%|██████████| 1/1 [00:00<00:00, 120.90it/s]

100%|██████████| 1/1 [00:00<00:00, 115.62it/s]

100%|██████████| 1/1 [00:00<00:00, 136.25it/s]

100%|██████████| 1/1 [00:00<00:00, 142.00it/s]

100%|██████████| 1/1 [00:00<00:00, 135.60it/s]

100

100%|██████████| 1/1 [00:00<00:00, 149.22it/s]

100%|██████████| 1/1 [00:00<00:00, 145.54it/s]

100%|██████████| 1/1 [00:00<00:00, 151.30it/s]

100%|██████████| 1/1 [00:00<00:00, 125.24it/s]

100%|██████████| 1/1 [00:00<00:00, 139.54it/s]

100%|██████████| 1/1 [00:00<00:00, 105.19it/s]

100%|██████████| 1/1 [00:00<00:00, 101.31it/s]

100%|██████████| 1/1 [00:00<00:00, 107.80it/s]

100%|██████████| 1/1 [00:00<00:00, 95.23it/s]

100%|██████████| 1/1 [00:00<00:00, 131.94it/s]

100%|██████████| 1/1 [00:00<00:00, 110.00it/s]

100%|██████████| 1/1 [00:00<00:00, 137.48it/s]

100%|██████████| 1/1 [00:00<00:00, 113.77it/s]

100%|██████████| 1/1 [00:00<00:00, 137.20it/s]

100%|██████████| 1/1 [00:00<00:00, 144.56it/s]

100%|██████████| 1/1 [00:00<00:00, 134.96it/s]

100%|██████████| 1/1 [00:00<00:00, 133.09it/s]

100%|██████████| 1/1 [00:00<00:00, 135.66it/s]

100%|██████████| 1/1 [00:00<00:00, 145.43it/s]

100%|██████████| 1/1 [00:00<00:00, 144.18it/s]

100%|██████████| 1/1 [00:00<00:00, 143.53

100%|██████████| 1/1 [00:00<00:00, 146.27it/s]

100%|██████████| 1/1 [00:00<00:00, 135.11it/s]

100%|██████████| 1/1 [00:00<00:00, 135.18it/s]

100%|██████████| 1/1 [00:00<00:00, 145.98it/s]

100%|██████████| 1/1 [00:00<00:00, 138.97it/s]

100%|██████████| 1/1 [00:00<00:00, 130.94it/s]

100%|██████████| 1/1 [00:00<00:00, 136.76it/s]

100%|██████████| 1/1 [00:00<00:00, 126.84it/s]

100%|██████████| 1/1 [00:00<00:00, 140.02it/s]

100%|██████████| 1/1 [00:00<00:00, 98.96it/s]

100%|██████████| 1/1 [00:00<00:00, 144.96it/s]

100%|██████████| 1/1 [00:00<00:00, 142.39it/s]

100%|██████████| 1/1 [00:00<00:00, 136.36it/s]

100%|██████████| 1/1 [00:00<00:00, 136.44it/s]

100%|██████████| 1/1 [00:00<00:00, 92.40it/s]

100%|██████████| 1/1 [00:00<00:00, 138.70it/s]

100%|██████████| 1/1 [00:00<00:00, 133.64it/s]

100%|██████████| 1/1 [00:00<00:00, 139.01it/s]

100%|██████████| 1/1 [00:00<00:00, 145.70it/s]

100%|██████████| 1/1 [00:00<00:00, 132.15it/s]

100%|██████████| 1/1 [00:00<00:00, 112.86i

100%|██████████| 1/1 [00:00<00:00, 142.98it/s]

100%|██████████| 1/1 [00:00<00:00, 147.89it/s]

100%|██████████| 1/1 [00:00<00:00, 158.45it/s]

100%|██████████| 1/1 [00:00<00:00, 147.09it/s]

100%|██████████| 1/1 [00:00<00:00, 139.20it/s]

100%|██████████| 1/1 [00:00<00:00, 141.10it/s]

100%|██████████| 1/1 [00:00<00:00, 163.80it/s]

100%|██████████| 1/1 [00:00<00:00, 143.60it/s]

100%|██████████| 1/1 [00:00<00:00, 150.71it/s]

100%|██████████| 1/1 [00:00<00:00, 125.86it/s]

100%|██████████| 1/1 [00:00<00:00, 125.61it/s]

100%|██████████| 1/1 [00:00<00:00, 137.86it/s]

100%|██████████| 1/1 [00:00<00:00, 141.86it/s]

100%|██████████| 1/1 [00:00<00:00, 141.57it/s]

100%|██████████| 1/1 [00:00<00:00, 131.01it/s]

100%|██████████| 1/1 [00:00<00:00, 102.31it/s]

100%|██████████| 1/1 [00:00<00:00, 149.29it/s]

100%|██████████| 1/1 [00:00<00:00, 80.17it/s]

100%|██████████| 1/1 [00:00<00:00, 140.38it/s]

100%|██████████| 1/1 [00:00<00:00, 135.30it/s]

100%|██████████| 1/1 [00:00<00:00, 134.67

100%|██████████| 1/1 [00:00<00:00, 131.30it/s]

100%|██████████| 1/1 [00:00<00:00, 140.61it/s]

100%|██████████| 1/1 [00:00<00:00, 131.37it/s]

100%|██████████| 1/1 [00:00<00:00, 135.14it/s]

100%|██████████| 1/1 [00:00<00:00, 137.56it/s]

100%|██████████| 1/1 [00:00<00:00, 140.72it/s]

100%|██████████| 1/1 [00:00<00:00, 133.54it/s]

100%|██████████| 1/1 [00:00<00:00, 137.50it/s]

100%|██████████| 1/1 [00:00<00:00, 147.77it/s]

100%|██████████| 1/1 [00:00<00:00, 109.66it/s]

100%|██████████| 1/1 [00:00<00:00, 137.44it/s]

100%|██████████| 1/1 [00:00<00:00, 142.42it/s]

100%|██████████| 1/1 [00:00<00:00, 144.25it/s]

100%|██████████| 1/1 [00:00<00:00, 142.79it/s]

100%|██████████| 1/1 [00:00<00:00, 137.25it/s]

100%|██████████| 1/1 [00:00<00:00, 133.85it/s]

100%|██████████| 1/1 [00:00<00:00, 137.72it/s]

100%|██████████| 1/1 [00:00<00:00, 136.95it/s]

100%|██████████| 1/1 [00:00<00:00, 138.79it/s]

100%|██████████| 1/1 [00:00<00:00, 109.34it/s]

100%|██████████| 1/1 [00:00<00:00, 118.6

In [14]:
len(embedding_list)

644800

In [26]:
# abstract가 'dummy'인 row (오류 생긴 부분)
error_idx = []

for i, emb in enumerate(embedding_list):
    if emb == 'dummy':
        error_idx.append(i)
        print(i)
        print(emb)
        print(abstracts[i])
        print('-'*100)

176342
dummy


----------------------------------------------------------------------------------------------------
258365
dummy
  
----------------------------------------------------------------------------------------------------
265289
dummy
  
----------------------------------------------------------------------------------------------------
309179
dummy
  
----------------------------------------------------------------------------------------------------
358188
dummy


----------------------------------------------------------------------------------------------------
358464
dummy


----------------------------------------------------------------------------------------------------
379793
dummy


----------------------------------------------------------------------------------------------------
395120
dummy
  
----------------------------------------------------------------------------------------------------


In [36]:
# abstract가 'dummy'인 row는 다 zero vector로 바꿈 (768차원)
for i in error_idx:
    embedding_list[i] = torch.zeros(768)

In [39]:
# Tensor의 리스트를 하나의 tensor로
stacked = torch.stack(embedding_list)

In [82]:
print(type(stacked))
print(stacked.shape)

<class 'torch.Tensor'>
torch.Size([644800, 768])


In [74]:
# Normalize
stacked_normalized = torch.nn.functional.normalize(stacked)

In [79]:
# torch.Tensor를 numpy.ndarray로
# FAISS 쓰려면 NumPy arrays 형태여야
stacked_np = stacked_normalized.numpy()

In [83]:
print(type(stacked_np))
print(stacked_np.shape)

<class 'numpy.ndarray'>
(644800, 768)


In [91]:
# numpy array 저장
np.save('abstract_embedding.npy', stacked_np) 

# 기존

In [1]:
import pandas as pd
import nltk
from nltk import word_tokenize, sent_tokenize
import torch
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('output.csv')

In [3]:
len(df)

725307

In [4]:
df.head()

,Unnamed: 0,id,title,venue,year,keywords,fos,refer_doc,n_citation,doc_type,abstract
0,0,53e99804b7602d9702016027,Functional logic overloading,Special Interest Group on Programming Languages,2002,"['functional logic programming', 'computationa...",Computational complexity theory,"['53e9a448b7602d9702d67a5d', '53e9a841b7602d97...",34,Conference,Functional logic overloading is a novel approa...
1,1,53e99dd4b7602d97026981ea,Modelling phenomena and dynamic logic of pheno...,Journal of Applied Non-Classical Logics,2012,"['field theory', 'computational complexity', '...",Computational complexity theory,['5c79247d4895d9cbc6239198'],12,Journal,Modeling of complex phenomena such as the mind...
2,2,53e9a2c0b7602d9702bc934f,Efficient algorithm for estimation of qualitat...,CoRR,2012,"['expected utility', 'case base reasoning']",Computational complexity theory,"['53e997c6b7602d9701fb94fc', '53e99818b7602d97...",6,Journal,We propose an e-cient algorithm for esti- mati...
3,3,53e9ab55b7602d97034e9f8b,Brief Fast spline smoothing via spectral facto...,Automatica (Journal of IFAC),2000,"['asymptotic degree', 'Brief Fast spline', 'co...",Computational complexity theory,"['53e9a3e7b7602d9702d004f6', '53e9a9f0b7602d97...",17,Journal,When tuning the smoothness parameter of nonpar...
4,4,53e9ae1cb7602d970382b83e,Independent component analysis with optimized ...,Antalya,2005,"['blind source separation', 'computational com...",Computational complexity theory,"['53e99940b7602d970217e09f', '53e99a30b7602d97...",1,Conference,The present contribution investigates the solu...


In [40]:
for i in range(10):
    print(df.iloc[i]['title'])
    print(df.iloc[i]['abstract'])
    print('# sentences:', len(sent_tokenize(df.iloc[i]['abstract'])))
    print('-'*100)

Functional logic overloading
Functional logic overloading is a novel approach to user-defined overloading that extends Haskell's concept of type classes in significant ways. Whereas type classes are conceptually predicates on types in standard Haskell, they are type functions in our approach. Thus, we can base type inference on the evaluation of functional logic programs. Functional logic programming provides a solid theoretical foundation for type functions and, at the same time, allows for programmable overloading resolution strategies by choosing different evaluation strategies for functional logic programs. Type inference with type functions is an instance of type inference with constrained types, where the underlying constraint system is defined by a functional logic program. We have designed a variant of Haskell which supports our approach to overloading, and implemented a prototype front-end for the language.


# sentences: 6
-----------------------------------------------------

## 추가 pre-processing
- nan인 abstract 제거

In [55]:
nan_idx = []

In [3]:
abstracts_all = list(df['abstract'])
len(abstracts_all)

725307

In [56]:
# abstract가 nan인 row
for i, abstract in enumerate(abstracts_all):
    if not isinstance(abstract, str):
        nan_idx.append(i)
        print(i)
        print(abstract)

1137
nan
1292
nan
9638
nan
117937
nan
275097
nan
528869
nan
584880
nan
606860
nan
615245
nan
618814
nan
626226
nan
629644
nan


In [57]:
nan_idx

[1137,
 1292,
 9638,
 117937,
 275097,
 528869,
 584880,
 606860,
 615245,
 618814,
 626226,
 629644]

In [6]:
type(abstracts_all[1137])

float

In [5]:
# abstract list에서 삭제
abstracts = [a for a in abstracts_all if isinstance(a, str)]
len(abstracts)

725295

In [86]:
df.filter(items=nan_idx, axis=0)['abstract']

1137      NaN
1292      NaN
9638      NaN
117937    NaN
275097    NaN
528869    NaN
584880    NaN
606860    NaN
615245    NaN
618814    NaN
626226    NaN
629644    NaN
Name: abstract, dtype: object

In [7]:
# df에서 삭제
df2 = df.filter(items=list(set(list(range(len(df)))) - set(nan_idx)), axis=0).drop(columns=['Unnamed: 0']).reset_index(drop=True)
df2

,id,title,venue,year,keywords,fos,refer_doc,n_citation,doc_type,abstract
0,53e99804b7602d9702016027,Functional logic overloading,Special Interest Group on Programming Languages,2002,"['functional logic programming', 'computationa...",Computational complexity theory,"['53e9a448b7602d9702d67a5d', '53e9a841b7602d97...",34,Conference,Functional logic overloading is a novel approa...
1,53e99dd4b7602d97026981ea,Modelling phenomena and dynamic logic of pheno...,Journal of Applied Non-Classical Logics,2012,"['field theory', 'computational complexity', '...",Computational complexity theory,['5c79247d4895d9cbc6239198'],12,Journal,Modeling of complex phenomena such as the mind...
2,53e9a2c0b7602d9702bc934f,Efficient algorithm for estimation of qualitat...,CoRR,2012,"['expected utility', 'case base reasoning']",Computational complexity theory,"['53e997c6b7602d9701fb94fc', '53e99818b7602d97...",6,Journal,We propose an e-cient algorithm for esti- mati...
3,53e9ab55b7602d97034e9f8b,Brief Fast spline smoothing via spectral facto...,Automatica (Journal of IFAC),2000,"['asymptotic degree', 'Brief Fast spline', 'co...",Computational complexity theory,"['53e9a3e7b7602d9702d004f6', '53e9a9f0b7602d97...",17,Journal,When tuning the smoothness parameter of nonpar...
4,53e9ae1cb7602d970382b83e,Independent component analysis with optimized ...,Antalya,2005,"['blind source separation', 'computational com...",Computational complexity theory,"['53e99940b7602d970217e09f', '53e99a30b7602d97...",1,Conference,The present contribution investigates the solu...
...,...,...,...,...,...,...,...,...,...,...
725290,5e72341193d709897cfba78b,Reachability Preserving Compression for Dynami...,Information Sciences,2020,"['Query preserving graph compression', 'Compre...",Mathematics,"['57d0637cac44367354287a89', '599c79e8601a182c...",3,Journal,Reachability preserving compression generates ...
725291,5e72343593d709897cfbefff,On the detection of selfish mining and stalker...,Annals of Telecommunications,2020,"['Blockchain', ' Selfish mining', ' Stalker at...",Mathematics,"['5550446145ce0a409eb4d18f', '5c8f64554895d9cb...",0,Journal,The blockchain technology emerged in 2008 as a...
725292,5e72344793d709897cfc1708,Related machine scheduling with machine speeds...,Journal of Combinatorial Optimization,2022,"['Related machine scheduling', ' Linear progra...",Mathematics,"['53e9b5dab7602d97041278ed', '5736960b6e3b1202...",0,Journal,We propose a related machine scheduling proble...
725293,5e72347b93d709897cfc85f5,Multiple conclusion linear logic: cut eliminat...,Journal of Logic and Computation,2020,"['full intuitionistic linear logic', 'classica...",Mathematics,"['53e99da4b7602d970265fa52', '53e9ad6db7602d97...",7,Journal,Full intuitionistic linear logic (FILL) was fi...


## SimCSE

In [6]:
from simcse import SimCSE
model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")

In [ ]:
%%time
embedding_list = []
# for i, abstract in tqdm(enumerate(abstracts)):
for i, abstract in enumerate(tqdm(abstracts)):
    try:
        sent_emb_list = model.encode(sent_tokenize(abstract), device="cuda", max_length=256)
        embedding_list.append(torch.mean(sent_emb_list, dim=0)) # 각 문장의 임베딩의 평균 (torch.Size([768]))
    except:
        print(f'Error: {i}')
        print(abstract)
        embedding_list.append('dummy')

In [23]:
# 11616/725295번째에서 에러 발생
abstracts[11616]

'\xa0\xa0'

In [33]:
len(embedding_list)

725295

In [60]:
# abstract가 'dummy'인 row (오류 생긴 부분)
error_idx = []

for i, emb in enumerate(embedding_list):
    if emb == 'dummy':
        error_idx.append(i)
        print(i)
        print(emb)
        print(abstracts[i])
        print('-'*100)

11616
dummy
  
----------------------------------------------------------------------------------------------------
80569
dummy
  
----------------------------------------------------------------------------------------------------
119134
dummy
  
----------------------------------------------------------------------------------------------------
128444
dummy
  
----------------------------------------------------------------------------------------------------
158836
dummy
  
----------------------------------------------------------------------------------------------------
165532
dummy
  
----------------------------------------------------------------------------------------------------
190474
dummy
  
----------------------------------------------------------------------------------------------------
469201
dummy


----------------------------------------------------------------------------------------------------
510046
dummy
  
---------------------------------------------------

In [61]:
print(len(error_idx))
print(error_idx)

31
[11616, 80569, 119134, 128444, 158836, 165532, 190474, 469201, 510046, 510806, 517206, 519145, 542396, 551131, 559402, 559714, 561820, 568096, 569738, 578099, 579793, 588190, 597536, 598341, 623117, 636079, 679533, 679874, 701277, 716865, 722823]


In [34]:
# embedding list에서 삭제
embedding_list_2 = [e for e in embedding_list if e!='dummy']

In [35]:
len(embedding_list_2)

725264

In [41]:
# 모두 768차원인 거 확인
for e in embedding_list_2:
    if e.shape[0] != 768:
        print('error')

In [87]:
df2.filter(items=error_idx, axis=0)['abstract']

11616       
80569       
119134      
128444      
158836      
165532      
190474      
469201    \n
510046      
510806      
517206      
519145      
542396      
551131      
559402      
559714      
561820      
568096      
569738      
578099      
579793      
588190      
597536      
598341      
623117      
636079      
679533    \n
679874    \n
701277    \n
716865      
722823      
Name: abstract, dtype: object

In [11]:
# df에서 삭제
df3 = df2.filter(items=list(set(list(range(len(df2)))) - set(error_idx)), axis=0).reset_index(drop=True)
df3

,id,title,venue,year,keywords,fos,refer_doc,n_citation,doc_type,abstract
0,53e99804b7602d9702016027,Functional logic overloading,Special Interest Group on Programming Languages,2002,"['functional logic programming', 'computationa...",Computational complexity theory,"['53e9a448b7602d9702d67a5d', '53e9a841b7602d97...",34,Conference,Functional logic overloading is a novel approa...
1,53e99dd4b7602d97026981ea,Modelling phenomena and dynamic logic of pheno...,Journal of Applied Non-Classical Logics,2012,"['field theory', 'computational complexity', '...",Computational complexity theory,['5c79247d4895d9cbc6239198'],12,Journal,Modeling of complex phenomena such as the mind...
2,53e9a2c0b7602d9702bc934f,Efficient algorithm for estimation of qualitat...,CoRR,2012,"['expected utility', 'case base reasoning']",Computational complexity theory,"['53e997c6b7602d9701fb94fc', '53e99818b7602d97...",6,Journal,We propose an e-cient algorithm for esti- mati...
3,53e9ab55b7602d97034e9f8b,Brief Fast spline smoothing via spectral facto...,Automatica (Journal of IFAC),2000,"['asymptotic degree', 'Brief Fast spline', 'co...",Computational complexity theory,"['53e9a3e7b7602d9702d004f6', '53e9a9f0b7602d97...",17,Journal,When tuning the smoothness parameter of nonpar...
4,53e9ae1cb7602d970382b83e,Independent component analysis with optimized ...,Antalya,2005,"['blind source separation', 'computational com...",Computational complexity theory,"['53e99940b7602d970217e09f', '53e99a30b7602d97...",1,Conference,The present contribution investigates the solu...
...,...,...,...,...,...,...,...,...,...,...
725259,5e72341193d709897cfba78b,Reachability Preserving Compression for Dynami...,Information Sciences,2020,"['Query preserving graph compression', 'Compre...",Mathematics,"['57d0637cac44367354287a89', '599c79e8601a182c...",3,Journal,Reachability preserving compression generates ...
725260,5e72343593d709897cfbefff,On the detection of selfish mining and stalker...,Annals of Telecommunications,2020,"['Blockchain', ' Selfish mining', ' Stalker at...",Mathematics,"['5550446145ce0a409eb4d18f', '5c8f64554895d9cb...",0,Journal,The blockchain technology emerged in 2008 as a...
725261,5e72344793d709897cfc1708,Related machine scheduling with machine speeds...,Journal of Combinatorial Optimization,2022,"['Related machine scheduling', ' Linear progra...",Mathematics,"['53e9b5dab7602d97041278ed', '5736960b6e3b1202...",0,Journal,We propose a related machine scheduling proble...
725262,5e72347b93d709897cfc85f5,Multiple conclusion linear logic: cut eliminat...,Journal of Logic and Computation,2020,"['full intuitionistic linear logic', 'classica...",Mathematics,"['53e99da4b7602d970265fa52', '53e9ad6db7602d97...",7,Journal,Full intuitionistic linear logic (FILL) was fi...


In [12]:
df3.to_csv('output_v2.csv', index=False)

In [105]:
# Tensor의 리스트를 하나의 tensor로
torch.stack(embedding_list_2) # torch.Size([725264, 768])

tensor([[ 0.0212, -0.0319,  0.0706,  ..., -0.0192,  0.0024, -0.0736],
        [ 0.0082, -0.0167,  0.0344,  ..., -0.0179, -0.0125, -0.0340],
        [ 0.0029, -0.0167,  0.0332,  ..., -0.0557, -0.0325, -0.0242],
        ...,
        [-0.0070, -0.0142,  0.0477,  ..., -0.0336, -0.0330, -0.0483],
        [-0.0008, -0.0058,  0.0354,  ..., -0.0011, -0.0442, -0.0817],
        [ 0.0048,  0.0223,  0.0464,  ..., -0.0170, -0.0084, -0.0481]])

In [109]:
%%time
torch.save(torch.stack(embedding_list_2), 'abstract_embedding.pt')

CPU times: user 1.22 s, sys: 1.83 s, total: 3.06 s
Wall time: 3.05 s
